In [1]:
import mediapipe as mp
import numpy as np
import cv2
import uuid
import os


In [9]:
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
mp_mesh=mp.solutions.face_mesh
mp_hands=mp.solutions.hands
joint_list=[[4,3,2],[8,7,6],[12,11,10],[16,15,14],[20,19,18]]

In [3]:
def getLabel(index,hand,results):
    output=None
    for idx,classification in enumerate(results.multi_handedness):
        
        if classification.classification[0].index==index:
            label=classification.classification[0].label
            score=classification.classification[0].score
            text='{} {}'.format(label,round(score,2))
            
            coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                [640,480]).astype(int)
            )
            output=text,coords
    return output



            

In [18]:

cap=cv2.VideoCapture(0)
with mp_mesh.FaceMesh(min_detection_confidence=0.8,max_num_faces=4) as facemesh:
    with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=4) as hands:
        with mp_pose.Pose(min_detection_confidence=0.8,min_tracking_confidence=0.5) as poses:
        
            while cap.isOpened():
                ret,frame=cap.read()
                image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                image=cv2.flip(image,1)
                image.flags.writeable=False
                results=facemesh.process(image) 
                res=hands.process(image)
                r=poses.process(image)
                image.flags.writeable=True
                image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
                if r.pose_landmarks:
                    mp_drawing.draw_landmarks(image,r.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                              mp_drawing.DrawingSpec(color=(180,180,180),thickness=1,circle_radius=1),
                                              mp_drawing.DrawingSpec(color=(180,180,180),thickness=1,circle_radius=1),
                                             )
                
                
                
                if results.multi_face_landmarks:
                    for num,face in enumerate(results.multi_face_landmarks):
                        mp_drawing.draw_landmarks(image,face,mp_mesh.FACE_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(180,180,180),thickness=1,circle_radius=1),
                                                  mp_drawing.DrawingSpec(color=(180,180,180),thickness=1,circle_radius=1),
                                                 )
                
                
                if res.multi_hand_landmarks:
                
                    for num,hand in enumerate(res.multi_hand_landmarks):
                        mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(190,190,190),thickness=2,circle_radius=2),
                                                  mp_drawing.DrawingSpec(color=(190,190,190),thickness=2,circle_radius=2),
                                                 )
                        if getLabel(num,hand,res):
                            text,coord=getLabel(num,hand,res)
                            cv2.putText(image,text,coord,cv2.FONT_HERSHEY_DUPLEX,1,(190,190,190),2,cv2.LINE_AA)
                #cv2.imwrite(os.path.join('Output','{}.jpg'.format(uuid.uuid1())),image)
                cv2.imshow('Hand Tracking',image)  
        
    
                if cv2.waitKey(1) & 0xFF==ord('q'):
                    break
        
cap.release()
cv2.destroyAllWindows()